# Azure AI Agents з підтримкою протоколу контексту моделі (MCP) - Python

Цей блокнот демонструє, як використовувати Azure AI Agents з інструментами протоколу контексту моделі (MCP) у Python. Він показує, як створити інтелектуального агента, який може використовувати зовнішні MCP сервери (наприклад, Microsoft Learn) для розширених можливостей за допомогою безключової автентифікації.


## Встановлення необхідних Python-пакетів

Спочатку потрібно встановити необхідні Python-пакети:
- **azure-ai-projects**: Основний SDK для Azure AI Projects
- **azure-ai-agents**: SDK для Azure AI Agents, який використовується для створення та управління агентами
- **azure-identity**: Забезпечує безключову автентифікацію за допомогою DefaultAzureCredential
- **mcp**: Реалізація протоколу Model Context Protocol для Python


## Переваги аутентифікації без ключів

Цей блокнот демонструє **аутентифікацію без ключів**, яка має кілька переваг:
- ✅ **Немає необхідності керувати API-ключами** - Використовує аутентифікацію на основі ідентичності Azure
- ✅ **Покращена безпека** - Секрети не зберігаються в коді або конфігураційних файлах
- ✅ **Автоматична ротація облікових даних** - Azure керує життєвим циклом облікових даних
- ✅ **Контроль доступу на основі ролей** - Використовує Azure RBAC для детального налаштування дозволів
- ✅ **Підтримка багатьох середовищ** - Працює безперешкодно як у середовищі розробки, так і у виробничому середовищі

`DefaultAzureCredential` автоматично обирає найкраще доступне джерело облікових даних:
1. **Керована ідентичність** (під час роботи в Azure)
2. Облікові дані **Azure CLI** (під час локальної розробки)
3. Облікові дані **Visual Studio**
4. **Змінні середовища** (якщо налаштовані)
5. Аутентифікація через **інтерактивний браузер** (як резервний варіант)


## Налаштування аутентифікації без ключів

**Передумови для аутентифікації без ключів:**

### Для локальної розробки:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Для середовищ Azure:
- Увімкніть **Системно призначену керовану ідентичність** на вашому ресурсі Azure
- Призначте відповідні **RBAC ролі** для керованої ідентичності:
  - `Cognitive Services OpenAI User` для доступу до Azure OpenAI
  - `AI Developer` для доступу до проектів Azure AI

### Змінні середовища (необов’язково):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Жодних API ключів або рядків підключення не потрібно!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Імпорт необхідних бібліотек

Імпортуйте необхідні модулі Python:
- **os, time**: Стандартні бібліотеки Python для змінних середовища та затримок
- **AIProjectClient**: Основний клієнт для Azure AI Projects
- **DefaultAzureCredential**: Безключова автентифікація для сервісів Azure
- **Класи, пов’язані з MCP**: Для створення та управління інструментами MCP і обробки затверджень


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Налаштування параметрів сервера MCP

Налаштуйте конфігурацію сервера MCP за допомогою змінних середовища з резервними значеннями за замовчуванням:
- **MCP_SERVER_URL**: URL сервера MCP (за замовчуванням використовується API Microsoft Learn)
- **MCP_SERVER_LABEL**: Мітка для ідентифікації сервера MCP (за замовчуванням "mslearn")

Такий підхід забезпечує гнучке налаштування для різних середовищ.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Створення клієнта проекту Azure AI (Безключова автентифікація)

Ініціалізуйте клієнта проекту Azure AI, використовуючи **безключову автентифікацію**:
- **endpoint**: URL-адреса кінцевої точки проекту Azure AI Foundry
- **credential**: Використовує `DefaultAzureCredential()` для безпечної, безключової автентифікації
- **Без необхідності API-ключів**: Автоматично знаходить і використовує найкращі доступні облікові дані

**Процес автентифікації:**
1. Перевіряє наявність Керованої Ідентичності (в середовищах Azure)
2. Переходить до облікових даних Azure CLI (для локальної розробки)
3. Використовує інші доступні джерела облікових даних за потреби

Цей підхід усуває необхідність керувати API-ключами або рядками підключення у вашому коді.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Створення визначення інструменту MCP

Створіть інструмент MCP, який підключається до сервера Microsoft Learn MCP:
- **server_label**: Ідентифікатор сервера MCP
- **server_url**: URL-адреса кінцевої точки сервера MCP
- **allowed_tools**: Необов’язковий список для обмеження інструментів, які можна використовувати (порожній список дозволяє всі інструменти)

Цей інструмент дозволить агенту отримувати доступ до документації та ресурсів Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Створення агента та виконання розмови (Безключовий робочий процес)

Цей розділ детально демонструє повний **безключовий робочий процес агента**:

1. **Створення AI-агента**: Налаштуйте агента з моделлю GPT-4.1 nano та інструментами MCP
2. **Створення потоку**: Встановіть потік розмови для спілкування
3. **Надсилання повідомлення**: Запитайте агента про відмінності між Azure OpenAI та OpenAI
4. **Обробка затверджень інструментів**: Автоматично затверджуйте виклики інструментів MCP, коли це необхідно
5. **Моніторинг виконання**: Відстежуйте прогрес агента та виконуйте необхідні дії
6. **Відображення результатів**: Показуйте деталі розмови та використання інструментів

**Особливості безключового підходу:**
- ✅ **Без жорстко закодованих секретів** - Уся автентифікація здійснюється через ідентифікацію Azure
- ✅ **Безпека за замовчуванням** - Використовує контроль доступу на основі ролей
- ✅ **Спрощене розгортання** - Не потребує управління обліковими даними
- ✅ **Зручно для аудиту** - Увесь доступ відстежується через ідентифікацію Azure

Агент використовуватиме інструменти MCP для доступу до ресурсів Microsoft Learn з повною безпекою та без необхідності управління API-ключами.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
